Импортируем библиотеки

In [2]:

import pandas as pd
import numpy as np
import random
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import trange
from gym import spaces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from collections import deque

In [3]:
import kagglehub

Чтение датасета

In [4]:
# Download latest version
path = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/network-intrusion-dataset


In [5]:
webattacks = pd.read_csv("/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
ddos = pd.read_csv("/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
normal = pd.read_csv("/kaggle/input/network-intrusion-dataset/Monday-WorkingHours.pcap_ISCX.csv")

In [6]:

webattacks.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,BENIGN
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,BENIGN
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,BENIGN
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN


In [7]:

ddos.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [8]:

normal.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


Предобработка

Объединение датасетов с последующей подготовкой

In [9]:

webattacks["attack_type"] = "WebAttack"
ddos["attack_type"] = "DDoS"
normal["attack_type"] = "Normal"

df = pd.concat([webattacks, ddos, normal], ignore_index=True)
df.columns = df.columns.str.strip()

df["is_attack"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)

df = df.replace([np.inf, -np.inf], np.nan).dropna()

Отбираем признаки

In [10]:
features = [col for col in df.columns if col not in ["Label", "attack_type", "is_attack"]]
X = df[features]
y = df["is_attack"]

In [11]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Радзеление на выборки

In [12]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print("Форма обучающей выборки:", X_train.shape)
print("Форма тестовой выборки:", X_test.shape)
print("Баланс классов (обучение):", np.bincount(y_train))

Форма обучающей выборки: (740338, 78)
Форма тестовой выборки: (185085, 78)
Баланс классов (обучение): [636174 104164]


Среда RL

In [13]:
class TrafficEnv(gym.Env):
    def __init__(self, X, y):
        super(TrafficEnv, self).__init__()
        self.X = X
        self.y = y
        self.current_index = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X.shape[1],), dtype=np.float32)
        self.action_space = spaces.Discrete(2)

    def reset(self):
        self.current_index = 0
        return self.X[self.current_index]

    def step(self, action):
        true_label = self.y[self.current_index]

        # Награда: +1 за правильное действие, -1 за неправильное
        if action == true_label:
            reward = 1
        else:
            reward = -1

        self.current_index += 1
        done = self.current_index >= len(self.X)

        if not done:
            next_state = self.X[self.current_index]
        else:
            next_state = np.zeros_like(self.X[0])  # Пустое состояние в конце

        return next_state, reward, done, {}

    def render(self, mode="human"):
        pass

Реализация DQN агента

In [14]:
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# Параметры
state_dim = X_train.shape[1]
action_dim = 2
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
lr = 0.001
batch_size = 64
replay_memory = deque(maxlen=10000)

# Инициализация
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_net = DQN(state_dim, action_dim).to(device)
optimizer = optim.Adam(policy_net.parameters(), lr=lr)
loss_fn = nn.MSELoss()

# Функция действия (ε-greedy)
def select_action(state, epsilon):
    if np.random.rand() < epsilon:
        return random.randint(0, action_dim - 1)
    state = torch.FloatTensor(state).unsqueeze(0).to(device)
    with torch.no_grad():
        q_values = policy_net(state)
    return torch.argmax(q_values).item()


Обучение агента в среде

In [15]:
env = TrafficEnv(X_train, y_train.values)
num_episodes = 25
epsilon = 1.0

max_steps = 100

for episode in trange(num_episodes, desc="Обучение агента"):
    state = env.reset()
    total_reward = 0
    done = False
    step_count = 0

    while not done and step_count < max_steps:
        action = select_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        step_count += 1

        replay_memory.append((state, action, reward, next_state, done))
        state = next_state

        if len(replay_memory) >= batch_size:
            batch = random.sample(replay_memory, batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)

            states = np.array(states)
            next_states = np.array(next_states)
            actions = np.array(actions)
            rewards = np.array(rewards)
            dones = np.array(dones)

            states = torch.FloatTensor(states).to(device)
            actions = torch.LongTensor(actions).unsqueeze(1).to(device)
            rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
            next_states = torch.FloatTensor(next_states).to(device)
            dones = torch.BoolTensor(dones).unsqueeze(1).to(device)

            q_values = policy_net(states).gather(1, actions)
            next_q_values = policy_net(next_states).max(1)[0].unsqueeze(1)
            expected_q = rewards + gamma * next_q_values * (~dones)

            loss = loss_fn(q_values, expected_q.detach())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    print(f"Эпизод {episode+1}: Общая награда = {total_reward}, шагов: {step_count}, ε = {epsilon:.3f}")

Обучение агента:   4%|▍         | 1/25 [00:00<00:05,  4.39it/s]

Эпизод 1: Общая награда = -16, шагов: 100, ε = 0.995


Обучение агента:   8%|▊         | 2/25 [00:00<00:05,  4.42it/s]

Эпизод 2: Общая награда = -6, шагов: 100, ε = 0.990


Обучение агента:  12%|█▏        | 3/25 [00:00<00:05,  4.39it/s]

...

Эпизод 23: Общая награда = 8, шагов: 100, ε = 0.891


Обучение агента:  96%|█████████▌| 24/25 [00:06<00:00,  3.02it/s]

Эпизод 24: Общая награда = 10, шагов: 100, ε = 0.887


Обучение агента: 100%|██████████| 25/25 [00:06<00:00,  3.73it/s]

Эпизод 25: Общая награда = 16, шагов: 100, ε = 0.882


Обучение агента на тестовой выборке с последующей оценкой

In [16]:
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)

predictions = []

with torch.no_grad():
    for state in X_test_np:
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        q_values = policy_net(state_tensor)
        action = torch.argmax(q_values).item()
        predictions.append(action)

print("Classification Report:")
print(classification_report(y_test_np, predictions, target_names=["Benign", "Attack"]))

print("Confusion Matrix:")
print(confusion_matrix(y_test_np, predictions))

Classification Report:
              precision    recall  f1-score   support

      Benign       0.98      1.00      0.99    159044
      Attack       0.97      0.85      0.90     26041

    accuracy                           0.97    185085
   macro avg       0.97      0.92      0.94    185085
weighted avg       0.97      0.97      0.97    185085

Confusion Matrix:
[[158359    685]
 [  4012  22029]]
